# 🤗 x 🪾: Mandryl's Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library. The majority of this notebook was written by the team at Huggingface, with small updates to make it *actually work* from the team at Mandryl.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 3.5 hours on an NVIDIA A100** GPU. On less powerful GPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.

## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.
Lerobot version 0.4.4 is specifically used here as it has been validated to work for this notebook.


In [ ]:
!conda install ffmpeg=7.1.1 -c conda-forge
!pip install "https://github.com/huggingface/lerobot/archive/33cad37054c2b594ceba57463e8f11ee374fa93c.zip"

## Weights & Biases and HF Hub login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging, and HF Hub log-in to make sure the model is saved correctly.

In [ ]:
!wandb login

## Huggingface Hub Login
This cell logs you into you account on the Huggingface Hub. This step is **very important**. If you skip this step, your train won't upload to the hub and will error out just before it finishes. Although there shouldn't be a problem as checkpoints are enabled (see below), it is better to be safe than sorry.

How do you create a Huggingface Hub account and get your API key? Follow this link to create an account [here](https://huggingface.co/). Once an account is created, navigate to your user profile -> Settings -> Access Tokens. Copy your access token and place below.

In [ ]:
!huggingface-cli login

## Training and Hub constants

Set the following constants to match your Hugging Face username, dataset repo, model repo name, and job name. They are used in the training command below so you can change them in one place. **Recommended dataset**: `lerobot/svla_so100_pickplace` (set as default for `dataset_repo_id` below).

In [ ]:
HF_USER = "YOUR_HF_USERNAME"       # Your Hugging Face Hub username (for uploading the trained model)
dataset_repo_id = "lerobot/svla_so100_pickplace"  # Recommended dataset. For your own: f"{HF_USER}/{mydataset}"
mydataset = "YOUR_DATASET"         # Your dataset repo name when using your own (use with HF_USER in dataset_repo_id)
my_smolvla = "my_smolvla"         # Repo name for the trained model on the Hub
my_smolvla_training = "my_smolvla_training"  # Job name for logging and wandb

## Install SmolVLA dependencies

In [ ]:
!cd lerobot && pip install -e ".[smolvla]"

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.

Make sure to adjust the following arguments to your setup:

1.`--policy.path=lerobot/smolvla_base` uses the `smolvla_base` as the base policy / model here. Theoretically any available model can be used, but performance and training time will be far less optimal.

2. `--dataset.repo_id`: Set `dataset_repo_id` in the constants section above. **Recommended**: `lerobot/svla_so100_pickplace`. For your own dataset use `f"{HF_USER}/{mydataset}"` (e.g. `pepijn223/il_gym0`).

3. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

4. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

5. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

6. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

7. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

8. `--rename_map`
   When your dataset uses different observation key names than what the policy expects, you must use `--rename_map` to map them at training time.

Once training finishes, the final model will be available at your output directory, plus on the Hub. Make sure to download your outputs folder from the Colab workspace as it will disapear when you refresh the session. The model will still be available on the Hub.

In [ ]:
!cd /content/lerobot && lerobot-train \
  --policy.path=lerobot/smolvla_base \
  --policy.repo_id=$HF_USER/$my_smolvla \
  --dataset.repo_id=$dataset_repo_id \
  --batch_size=64 \
  --steps=20000 \
  --save_checkpoint=true \
  --save_freq=2000 \
  --output_dir=outputs/train/$my_smolvla \
  --job_name=$my_smolvla_training \
  --policy.device=cuda \
  --wandb.enable=true \
  --rename_map='{"observation.images.up": "observation.images.camera1", "observation.images.side": "observation.images.camera2"}'

Once the train finishes, that's it! View your dataset in the Hub and follow the provided instructions to run inference.